In [1]:
import os
import re
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import InputLayer,Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\anish\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\anish\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\anish\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "C:\Users\anish\anaconda3\Lib\site-packa

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [2]:
learning_rate=0.001
tf.config.run_functions_eagerly(True)

# Enable eager execution for tf.data functions
#tf.data.experimental.enable_debug_mode()


In [3]:
def load_dataset(dataset_dir):
    images = []
    labels = []
    label_mapping = {}  # Mapping matrix to store label-to-number mapping
    label_counter = 0   # Counter to assign numeric values to labels

    for filename in os.listdir(dataset_dir):
        if filename.endswith('.png') or filename.endswith('.jpg'):
            # Load image
            image = cv2.imread(os.path.join(dataset_dir, filename))
            # Convert to grayscale
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # Resize image to a consistent size
            resized_image = cv2.resize(gray_image, (100, 100))
            # Deblur the image
            deblurred_image = cv2.GaussianBlur(resized_image, (5, 5), 0)
            # Normalize pixel values to range [0, 1]
            normalized_image = deblurred_image / 255.0
            # Append image to list
            images.append(normalized_image)
            # Extract label from filename after the 17th character
            label = filename[17:].split('.')[0]  # Remove file extension
            # Check if the label already exists in the mapping matrix
            if label not in label_mapping:
                # Assign a numeric value to the label and update the mapping matrix
                label_mapping[label] = label_counter
                label_counter += 1
            # Append the numeric label to the labels list
            labels.append(label_mapping[label])

    return images, labels, label_mapping


In [4]:
images, labels, label_mapping = load_dataset(r"C:\Users\anish\Desktop\hand")
images = np.array(images).reshape(-1, 100, 100, 1)
labels = tf.keras.utils.to_categorical(labels, num_classes=len(label_mapping))


In [5]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.1, random_state=0)

In [6]:
# Data Augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,        # Degree range for random rotations
    width_shift_range=0.1,    # Fraction of total width for horizontal shifts
    height_shift_range=0.1,   # Fraction of total height for vertical shifts
    horizontal_flip=True     # Randomly flip inputs horizontally
)

In [7]:
datagen.fit(X_train)

In [8]:
model = Sequential([
    InputLayer(shape=(100, 100, 1)),  # Input layer
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    Flatten(),
    Dense(1000, activation='relu'),
    Dropout(0.5),
    Dense(len(label_mapping), activation='softmax')
])

In [9]:
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Train the model with augmented data
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=50, validation_data=(X_val, y_val))

Epoch 1/50


C:\Users\anish\anaconda3\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
C:\Users\anish\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


61/61 ━━━━━━━━━━━━━━━━━━━━ 60s 920ms/step - accuracy: 0.3794 - loss: 11.0399 - val_accuracy: 0.0512 - val_loss: 21.4351
Epoch 2/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 53s 864ms/step - accuracy: 0.7222 - loss: 4.6898 - val_accuracy: 0.0372 - val_loss: 102.7755
Epoch 3/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 52s 850ms/step - accuracy: 0.8007 - loss: 3.1456 - val_accuracy: 0.0372 - val_loss: 179.2235
Epoch 4/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 52s 842ms/step - accuracy: 0.8605 - loss: 2.2487 - val_accuracy: 0.0791 - val_loss: 163.8494
Epoch 5/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 51s 827ms/step - accuracy: 0.8755 - loss: 1.5711 - val_accuracy: 0.0791 - val_loss: 194.6902
Epoch 6/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 50s 822ms/step - accuracy: 0.8768 - loss: 1.1984 - val_accuracy: 0.0837 - val_loss: 201.6116
Epoch 7/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 53s 868ms/step - accuracy: 0.8930 - loss: 1.3229 - val_accuracy: 0.0884 - val_loss: 151.1543
Epoch 8/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 50s 819ms/step - accuracy: 0.8919 - loss: 1.2852 - val_a

In [11]:
# Print training and validation accuracy
train_acc = history.history['accuracy'][-1] * 100
val_acc = history.history['val_accuracy'][-1] * 100
print(f"Training Accuracy: {train_acc:.2f}%, Validation Accuracy: {val_acc:.2f}%")


Training Accuracy: 97.35%, Validation Accuracy: 96.74%


In [12]:
# Calculate evaluation metrics
y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)
y_train_pred_classes = np.argmax(y_train_pred, axis=1)
y_val_pred_classes = np.argmax(y_val_pred, axis=1)
train_accuracy = accuracy_score(np.argmax(y_train, axis=1), y_train_pred_classes)
val_accuracy = accuracy_score(np.argmax(y_val, axis=1), y_val_pred_classes)
precision = precision_score(np.argmax(y_val, axis=1), y_val_pred_classes, average='weighted')
recall = recall_score(np.argmax(y_val, axis=1), y_val_pred_classes, average='weighted')
f1 = f1_score(np.argmax(y_val, axis=1), y_val_pred_classes, average='weighted')

# Print evaluation metrics
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")

61/61 ━━━━━━━━━━━━━━━━━━━━ 6s 104ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step
Training Accuracy: 98.18%
Validation Accuracy: 96.74%
Precision: 97.16%
Recall: 96.74%
F1 Score: 96.70%


In [14]:
# Save the trained model in the native Keras format
model.save("TrainedModel/GestureRecogModel.keras")


In [15]:
loaded_model = tf.keras.models.load_model('TrainedModel/GestureRecogModel.keras')

In [23]:
# Load the single test image
test_image = cv2.imread(r"C:\Users\anish\Desktop\hand\frame_0000_hand_0TA.jpg")
test_image_gray = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
resized_test_image = cv2.resize(test_image_gray, (100, 100))
deblurred_test_image = cv2.GaussianBlur(resized_test_image, (5, 5), 0)
normalized_test_image = deblurred_test_image / 255.0

# Reshape the image for model input
test_image_input = normalized_test_image.reshape(1, 100, 100, 1)

# Predict on the single test image
prediction = loaded_model.predict(test_image_input)
predicted_class = np.argmax(prediction)

# Output the label of the predicted class
predicted_label = list(label_mapping.keys())[list(label_mapping.values()).index(predicted_class)]
print("Predicted class:", predicted_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Predicted class: TA
